In [1]:
import mxnet as mx
import math
import numpy as np

class DropExtract(mx.operator.CustomOp):
    def forward(self, is_train, req, in_data, out_data, aux):
        drop_in     = in_data[0].asnumpy()
        drop_cls    = (drop_in[:,0]>0.5).reshape(-1,1)
        drop_cls    = np.broadcast_to(drop_cls, (drop_cls.shape[0], 512))
        drop_out    = drop_in * drop_cls
        self.assign(out_data[0], req[0], mx.nd.array(drop_out))
    def backward(self, req, out_grad, in_data, out_data, in_grad, aux):
        self.assign(in_grad[0], req[0], 0)

@mx.operator.register("dropExtract")
class DropExtractProp(mx.operator.CustomOpProp):
    def __init__(self):
        super(DropExtractProp, self).__init__(need_top_grad=False)

    def list_arguments(self):
        return ['data']

    def list_outputs(self):
        return ['output']

    def infer_shape(self, in_shape):
        data_shape = in_shape[0]
        output_shape = in_shape[0]
        return [data_shape], [output_shape], []

    def create_operator(self, ctx, shapes, dtypes):
        return DropExtract()


In [5]:
data = mx.sym.Variable('data')
x = mx.nd.zeros((128,512))
x[0:100] = 2
custom=mx.symbol.Custom(data=data, name='drop_ex', op_type='dropExtract')

In [7]:
ex = custom.bind(ctx=mx.cpu(), args={'data':x})

In [8]:
ex.forward()

[<NDArray 128x512 @cpu(0)>]

In [11]:
ex.outputs[0].asnumpy()

array([[ 2.,  2.,  2., ...,  2.,  2.,  2.],
       [ 2.,  2.,  2., ...,  2.,  2.,  2.],
       [ 2.,  2.,  2., ...,  2.,  2.,  2.],
       ..., 
       [ 0.,  0.,  0., ...,  0.,  0.,  0.],
       [ 0.,  0.,  0., ...,  0.,  0.,  0.],
       [ 0.,  0.,  0., ...,  0.,  0.,  0.]], dtype=float32)

In [12]:
ex.backward()

In [13]:
ex.grad_arrays[0].wait_to_read()

TypeError: 'NoneType' object has no attribute '__getitem__'